# SPCS Networking Connectivity Test: GitHub

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

1. Please provide your GitHub organization or username
2. Check that the Network Rule structure matches your requirements
3. Run the Connectivity Test if your EAI is already configured
4. Use the Cells to prepare the EAI if needed, then rerun the test.

In [ ]:
# GitHub Connectivity Test Configuration

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# This role will be used by Openflow to run Connectors and perform your Data Engineering tasks
OPENFLOW_RUNTIME_ROLE = "OPENFLOWRUNTIMEROLE"

# For authentication testing (optional - leave as REPLACEME if not testing auth)
GITHUB_TOKEN = "REPLACEME"  # Replace with your GitHub Personal Access Token for auth testing

# GitHub API endpoints
GITHUB_API_URL = "https://api.github.com"
GITHUB_URL = "https://github.com"

print(f"Configuration:")
print(f"GitHub API: {GITHUB_API_URL}")
print(f"GitHub Site: {GITHUB_URL}")
print("\nReady to test connectivity...")

In [ ]:
import requests

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\n🔍 Testing {description}: {url}")
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  ✅ HTTP Response: {response.status_code}")
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  ❌ Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("GITHUB CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['github_api'] = test_connectivity(GITHUB_API_URL, "GitHub API")
results['github_site'] = test_connectivity(GITHUB_URL, "GitHub Site")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\n🎉 All connectivity tests PASSED!")
    print("Your SPCS environment can reach all required GitHub endpoints.")
    print("You can proceed with GitHub integration.")
else:
    print(f"\n⚠️  Some connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

## GitHub Authentication Test (Optional)

If you have a GitHub Personal Access Token, you can test authentication. This is optional - GitHub Apps use a different authentication flow.


In [ ]:
import requests

print("=" * 60)
print("GITHUB PAT AUTHENTICATION TEST")
print("=" * 60)

if not GITHUB_TOKEN or GITHUB_TOKEN == "REPLACEME":
    print("\n⚠️  No GitHub token provided - skipping authentication test")
    print("This is optional. GitHub Apps use a different authentication flow.")
else:
    print("\nTesting authentication with provided Personal Access Token...")

    try:
        headers = {
            'Authorization': f'token {GITHUB_TOKEN}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Test authenticated request
        response = requests.get(f"{GITHUB_API_URL}/user", headers=headers, timeout=10)

        if response.status_code == 200:
            user_data = response.json()
            print(f"\n✅ Authentication SUCCESSFUL!")
            print(f"   Authenticated as: {user_data.get('login', 'N/A')}")
            print(f"   Account type: {user_data.get('type', 'N/A')}")
        elif response.status_code == 401:
            print(f"\n❌ Authentication FAILED: Invalid or expired token")
        else:
            print(f"\n⚠️  Unexpected response: {response.status_code}")

    except Exception as e:
        print(f"\n❌ Error testing authentication: {e}")


## External Access Integration (EAI) Setup

If Connectivity testing fails, you can use the cells below to prepare and implement an EAI suitable for GitHub access.

After creating and setting the EAI, return to Cell 2 to run the connectivity test again, then run the API discovery cells to validate your credentials and discover repositories.

In [ ]:
-- Create Network Rules for GitHub connectivity
-- Run this cell if connectivity tests failed

USE ROLE {{IMPLEMENTATION_ROLE}};

-- Network rule for GitHub API
CREATE OR REPLACE NETWORK RULE github_api_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('api.github.com');

-- Network rule for GitHub site
CREATE OR REPLACE NETWORK RULE github_site_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('github.com');

SHOW NETWORK RULES LIKE 'github_%';

In [ ]:
-- Create External Access Integration for GitHub
-- This combines all the network rules into a single integration

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION github_eai
  ALLOWED_NETWORK_RULES = (
    github_api_rule,
    github_site_rule
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for GitHub API connectivity';

-- Grant usage on the integration to your roles
GRANT USAGE ON INTEGRATION github_eai TO ROLE {{IMPLEMENTATION_ROLE}};
GRANT USAGE ON INTEGRATION github_eai TO ROLE {{OPENFLOW_RUNTIME_ROLE}};

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'github_eai';

In [ ]:
ALTER NOTEBOOK EAI_GITHUB
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('GITHUB_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.